### Clipping of NetCDF to Shape File

Imports

In [1]:
import os,sys #user to create and modify file name and save it
from netCDF4 import Dataset#to read the .nc files
import pandas as pd #for dataframe activities
from datetime import datetime,timedelta #for the time related activities
import matplotlib.pyplot as plt #for the Ploting purpose
import numpy as np #numerical fucntions
import datetime 
import warnings #to ignore unnecessary warnings
import xarray as xr
import numpy as np
import regionmask
import geopandas as gp
import glob                                                                 
from pyhdf.SD import SD, SDC
from zmq import ctx_opt_names #for the Ploting purpose
warnings.filterwarnings('ignore')


Day of the year Calculator

In [ ]:
from datetime import datetime,date
d=date(2007,5,31)
dayofyear=d.timetuple().tm_yday
print(dayofyear)

Defining function to correct the taken dataset

In [3]:
def data_correction(DATAFIELD):
    data2D = hdf.select(DATAFIELD)
    data = data2D[:,:].astype(np.double)

    # Retrieve attributes.
    attrs = data2D.attributes(full=1)
    aoa=attrs["add_offset"]
    add_offset = aoa[0]
    fva=attrs["_FillValue"]
    _FillValue = fva[0]
    sfa=attrs["scale_factor"]
    scale_factor = sfa[0]        
    ua=attrs["units"]
    units = ua[0]
    data[data == _FillValue] = np.nan
    data = (data - add_offset) * scale_factor 
    datam = np.ma.masked_array(data, np.isnan(data))
    return datam


The loop to read all the files and clip them

In [ ]:
os.chdir('/Volumes/PtatoBasket/Datasets/Unclipped_MYD/PRE-MONSOON/2005-2006')#For changing the wokring directory
for file in list(glob.glob('MYD08*.hdf')):

    reader = open(file)
    hdf = SD(file, SDC.READ)

    # Read geolocation dataset.
    lat = hdf.select('YDim')
    latitude = lat[:]
    lon = hdf.select('XDim')
    longitude = lon[:]

    #Getting the dates from filename
    arr=str(file).split(".")
    year=str(arr[1])[1:5]
    days=str(arr[1])[5:]
    dat=datetime(int(year), 1, 1) + timedelta(int(days) - 1)
    dat=(dat-datetime(2005,1,1)).days
    
    #Callingout the pre-defined function for the correction
    cer = data_correction('Cloud_Effective_Radius_Liquid_Mean')
    cot=data_correction('Cloud_Optical_Thickness_Combined_Mean')
    ctt=data_correction('Cloud_Top_Temperature_Mean')
    ctp=data_correction('Cloud_Top_Pressure_Mean')

    # writing the ncfile
    df = xr.Dataset(
            data_vars={'Cloud_Effective_Radius_Liquid_Mean':    (('latitude', 'longitude' ), cer),
                        'Cloud_Optical_Thickness_Combined_Mean':(('latitude', 'longitude' ), cot),
                        'Cloud_Top_Temperature_Mean':(('latitude', 'longitude' ), ctt),
                        'Cloud_Top_Pressure_Mean':(('latitude', 'longitude' ), ctp),
                    },
            coords={'longitude': longitude,
                'latitude': latitude,
                'time':dat,
            })

    df.Cloud_Effective_Radius_Liquid_Mean.attrs['long_name'] = 'Cloud Effective Radius'
    df.Cloud_Optical_Thickness_Combined_Mean.attrs['long_name'] = 'Cloud Optical Radius'
    df.Cloud_Top_Temperature_Mean.attrs['long_name'] = 'Cloud Top Temperautre'
    df.Cloud_Top_Pressure_Mean.attrs['long_name'] = 'Cloud Top Pressure'

    # defining the coordinates
    df.longitude.attrs['standard_name'] = 'longitude'
    df.longitude.attrs['long_name'] = 'longitude'
    df.longitude.attrs['units'] = 'degrees_east'
    df.longitude.attrs['axis'] = 'X'

    df.latitude.attrs['standard_name'] = 'latitude'
    df.latitude.attrs['long_name'] = 'latitude'
    df.latitude.attrs['units'] = 'degrees_north'
    df.latitude.attrs['axis'] = 'Y'


    df.time.attrs['standard_name'] = 'time'
    df.time.attrs['long_name'] = 'time'
    df.time.attrs['axis'] = 'T'
    df.time.attrs["units"] = "days since 2005-01-01"

    #Opening the already created mask file
    mask=xr.open_dataarray('/Volumes/ACIML/Main/BOB_MASK.nc')
    masked_shape=df.where(mask==0)

    min_lon = 78.00
    max_lon = 96.00
    min_lat = 8.00
    max_lat = 23.00

    cropped_ds = masked_shape.sel(latitude=slice(max_lat,min_lat), longitude=slice(min_lon,max_lon))#Croppping the masked dataset to our range
    fname=str(file)[:-3]
    cropped_ds.to_netcdf('/Volumes/ACIML/Clipped-MOD/'+fname+'nc')#saving the final masked out

Reading the shape file and making a mask

In [ ]:
shapefile ="/Volumes/ACIML/Main/SHPs/BOB_OCEAN/BOB_OCEAN.shp"
countries=gp.read_file(shapefile,engine='pyogrio')

fig, ax =plt.subplots(figsize=(16,10))
countries.plot(ax=ax,column='featurecla') 
c_list=list(countries['featurecla'])
c_list_unique=set(list(countries['featurecla']))
indexes=[c_list.index(x) for x in c_list_unique]
countries_mask_poly=regionmask.Regions(outlines=countries.geometry[indexes],name='featurecla',numbers=indexes,names=countries.featurecla[indexes])
print("{}".format(countries_mask_poly.names[0]))

#Clipping to shape File and Creating Mask File
shapefile ="/Volumes/ACIML/Main/SHPs/BOB_OCEAN/BOB_OCEAN.shp"
countries=gp.read_file(shapefile,engine='pyogrio')
c_list=list(countries['featurecla'])
c_list_unique=set(list(countries['featurecla']))
indexes=[c_list.index(x) for x in c_list_unique]
countries_mask_poly=regionmask.Regions(outlines=countries.geometry[indexes],name='featurecla',numbers=indexes,names=countries.featurecla[indexes])
mask=countries_mask_poly.mask(df,lat_name='latitude',lon_name='longitude')
mask.to_netcdf('BOB_MASK.nc')